# 1.Multigpu Distributed Training-ScriptMode
---

본 모듈에서는 Amzaon SageMaker API을 효과적으로 이용하기 위해 multigpu-distributed 학습을 위한 PyTorch 프레임워크 자체 구현만으로 모델 훈련을 수행해 봅니다.

## 1. Sagemaker notebook 설명
<p>Sagemaker notebook은 완전 관리형 서비스로 컨테이너 기반으로 구성되어 있습니다. 사용자가 직접 컨테이너를 볼 수 없지만, 내부적으로는 아래와 같은 원리로 동작합니다. </p>
<p><img src="./imgs/fig00.png" width="800", height="80"></p>

- **S3 (Simple Storage Serivce)** : Object Storage로서 학습할 데이터 파일과 학습 결과인 model, checkpoint, tensorboard를 위한 event 파일, 로그 정보 등을 저장하는데 사용합니다.
- **SageMaker Notebook** : 학습을 위한 스크립트 작성과 디버깅, 그리고 실제 학습을 수행하기 위한 Python을 개발하기 위한 환경을 제공합니다.
- **Amazon Elastic Container Registry(ECR)** :  Docker 컨테이너 이미지를 손쉽게 저장, 관리 및 배포할 수 있게 해주는 완전관리형 Docker 컨테이너 레지스트리입니다. Sagemaker는 기본적인 컨테이너를 제공하기 때문에 별도 ECR에 컨테이너 이미지를 등록할 필요는 없습니다. 하지만, 별도의 학습 및 배포 환경이 필요한 경우 custom 컨테이너 이미지를 만들어서 ECR에 등록한 후 이 환경을 활용할 수 있습니다.

<p>학습과 추론을 하는 hosting 서비스는 각각 다른 컨테이너 환경에서 수행할 수 있으며, 쉽게 다량으로 컨테이너 환경을 확장할 수 있으므로 다량의 학습과 hosting이 동시에 가능합니다.   
</p>

In [1]:
import sys

In [2]:
!{sys.executable} -m pip install split-folders tqdm albumentations crc32c -U sagemaker
# !{sys.executable} -m pip install 'sagemaker[local]' --upgrade

Requirement already up-to-date: split-folders in /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages (0.4.3)
Requirement already up-to-date: tqdm in /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages (4.54.1)
Requirement already up-to-date: albumentations in /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages (0.5.2)
Requirement already up-to-date: crc32c in /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages (2.2)
Requirement already up-to-date: sagemaker in /home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages (2.19.0)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_latest_p36/bin/python -m pip install --upgrade pip' command.


## 2. 환경 설정

<p>Sagemaker 학습에 필요한 기본적인 package를 import 합니다. </p>
<p>boto3는 HTTP API 호출을 숨기는 편한 추상화 모델을 가지고 있고, Amazon EC2 인스턴스 및 S3 버켓과 같은 AWS 리소스와 동작하는 파이선 클래스를 제공합니다. </p>
<p>sagemaker python sdk는 Amazon SageMaker에서 기계 학습 모델을 교육 및 배포하기 위한 오픈 소스 라이브러리입니다.</p>

In [3]:
import joblib
import matplotlib.pyplot as plt
import sagemaker
import splitfolders

import datetime
import glob
import os
import time
import warnings

import boto3
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision

from PIL import Image
from torch.utils.data import Dataset
from torchvision import datasets, transforms

from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

In [4]:
sess = boto3.Session()
sagemaker_session = sagemaker.Session()
sm = sess.client('sagemaker')
role = get_execution_role()

## 3. S3의 저장 데이터 위치 가져오기
<p> 데이터를 정하기 위한 S3의 bucket 위치는 아래 data_bucket 이름으로 생성하며, 기본적으로 SageMaker에서 학습한 모델과 로그 정보를 남기는 위치는 자동으로 생성되는 bucket 이름으로 저장됩니다. </p>

In [5]:
# create a s3 bucket to hold data, note that your account might already created a bucket with the same name
account_id = sess.client('sts').get_caller_identity()["Account"]
job_bucket = 'sagemaker-experiments-{}-{}'.format(sess.region_name, account_id)
data_bucket = 'sagemaker-{}-{}'.format(sess.region_name, account_id)
try:
    if sess.region_name == "us-east-1":
        sess.client('s3').create_bucket(Bucket=data_bucket)
    else:
        sess.client('s3').create_bucket(Bucket=data_bucket, 
                                        CreateBucketConfiguration={'LocationConstraint': sess.region_name})
except Exception as e:
    print(e)

## 4. Dataset 소개 및 split 후 S3 upload하기
<p>이번 학습에 사용할 이미지 데이터는 <strong><a href="https://www.robots.ox.ac.uk/~vgg/data/pets/" target="_blank" class ='btn-default'>Oxford-IIIT Pet Dataset</a></strong> 입니다. Oxford-IIIT Pet Dataset은 <strong>37</strong>개 다른 종의 개와 고양이 이미지를 각각 200장 씩 제공하고 있으며, Ground Truth 또한 Classification, Object Detection, Segmentation와 관련된 모든 정보가 있으나, 이번 학습에서는 37개 class에 대해 일부 이미지로 Classification 문제를 해결하기 위해 학습을 진행할 예정입니다.</p>
<p><img src="./imgs/pet_annotations.jpg" width="700", height="70"></p>    
<p>이미지 파일을 학습하기 위해 SageMaker Notebook 환경으로 upload를 합니다. 폴더 구조는 아래와 같은 형태로 구성되어야 합니다. </p>
<pre>
<div style='line-height:80%'>
    image_path/class1/Aimage_1<br>
                      Aimage_2<br>
                       ...<br>
                      Aimage_N<br>
    image_path/class2/Bimage_1<br>
                      Bimage_2<br>
                       ...<br>
                      Bimage_M<br>
</div>
</pre>

<p>SageMaker 학습을 위해 train/val로 분리한 폴더를 S3내 이전에 지정한 data_bucket 내 prefix 하위 폴더로 upload합니다. </p>

In [6]:
!cat ./raw_img/data.tar.gz* | tar -zxvpf -

data/
data/Ragdoll/
data/Ragdoll/Ragdoll_171.jpg
data/Ragdoll/Ragdoll_78.jpg
data/Ragdoll/Ragdoll_163.jpg
data/Ragdoll/Ragdoll_257.jpg
data/Ragdoll/Ragdoll_43.jpg
data/Ragdoll/Ragdoll_254.jpg
data/Ragdoll/Ragdoll_193.jpg
data/Ragdoll/Ragdoll_4.jpg
data/Ragdoll/Ragdoll_175.jpg
data/Ragdoll/Ragdoll_18.jpg
data/Ragdoll/Ragdoll_199.jpg
data/Ragdoll/Ragdoll_104.jpg
data/Ragdoll/Ragdoll_37.jpg
data/Ragdoll/Ragdoll_3.jpg
data/Ragdoll/Ragdoll_75.jpg
data/Ragdoll/Ragdoll_96.jpg
data/Ragdoll/Ragdoll_44.jpg
data/Ragdoll/Ragdoll_80.jpg
data/Ragdoll/Ragdoll_137.jpg
data/Ragdoll/Ragdoll_90.jpg
data/Ragdoll/Ragdoll_100.jpg
data/Ragdoll/Ragdoll_25.jpg
data/Ragdoll/Ragdoll_135.jpg
data/Ragdoll/Ragdoll_210.jpg
data/Ragdoll/Ragdoll_98.jpg
data/Ragdoll/Ragdoll_180.jpg
data/Ragdoll/Ragdoll_74.jpg
data/Ragdoll/Ragdoll_161.jpg
data/Ragdoll/Ragdoll_185.jpg
data/Ragdoll/Ragdoll_255.jpg
data/Ragdoll/Ragdoll_62.jpg
data/Ragdoll/Ragdoll_68.jpg
data/Ragdoll/Ragdoll_179.jpg
data/Ragdoll/Ragdoll_121.jpg
data/Ragdoll

data/japanese_chin/japanese_chin_81.jpg
data/japanese_chin/japanese_chin_73.jpg
data/japanese_chin/japanese_chin_95.jpg
data/japanese_chin/japanese_chin_51.jpg
data/japanese_chin/japanese_chin_117.jpg
data/japanese_chin/japanese_chin_92.jpg
data/japanese_chin/japanese_chin_124.jpg
data/japanese_chin/japanese_chin_71.jpg
data/japanese_chin/japanese_chin_169.jpg
data/japanese_chin/japanese_chin_18.jpg
data/japanese_chin/japanese_chin_199.jpg
data/japanese_chin/japanese_chin_113.jpg
data/japanese_chin/japanese_chin_47.jpg
data/japanese_chin/japanese_chin_86.jpg
data/japanese_chin/japanese_chin_91.jpg
data/japanese_chin/japanese_chin_49.jpg
data/japanese_chin/japanese_chin_94.jpg
data/japanese_chin/japanese_chin_121.jpg
data/japanese_chin/japanese_chin_102.jpg
data/japanese_chin/japanese_chin_44.jpg
data/japanese_chin/japanese_chin_144.jpg
data/japanese_chin/japanese_chin_26.jpg
data/japanese_chin/japanese_chin_59.jpg
data/japanese_chin/japanese_chin_87.jpg
data/japanese_chin/japanese_chin

data/samoyed/samoyed_85.jpg
data/samoyed/samoyed_61.jpg
data/samoyed/samoyed_26.jpg
data/samoyed/samoyed_54.jpg
data/samoyed/samoyed_104.jpg
data/samoyed/samoyed_18.jpg
data/samoyed/samoyed_15.jpg
data/samoyed/samoyed_125.jpg
data/samoyed/samoyed_143.jpg
data/samoyed/samoyed_69.jpg
data/samoyed/samoyed_5.jpg
data/samoyed/samoyed_30.jpg
data/samoyed/samoyed_136.jpg
data/samoyed/samoyed_38.jpg
data/samoyed/samoyed_177.jpg
data/samoyed/samoyed_128.jpg
data/samoyed/samoyed_86.jpg
data/samoyed/samoyed_197.jpg
data/samoyed/samoyed_81.jpg
data/samoyed/samoyed_110.jpg
data/samoyed/samoyed_108.jpg
data/samoyed/samoyed_150.jpg
data/samoyed/samoyed_158.jpg
data/samoyed/samoyed_127.jpg
data/samoyed/samoyed_99.jpg
data/samoyed/samoyed_77.jpg
data/samoyed/samoyed_75.jpg
data/samoyed/samoyed_172.jpg
data/samoyed/samoyed_114.jpg
data/samoyed/samoyed_162.jpg
data/samoyed/samoyed_71.jpg
data/samoyed/samoyed_146.jpg
data/samoyed/samoyed_101.jpg
data/samoyed/samoyed_149.jpg
data/samoyed/samoyed_131.jpg
da

data/Persian/Persian_4.jpg
data/Persian/Persian_52.jpg
data/Persian/Persian_270.jpg
data/Persian/Persian_126.jpg
data/Persian/Persian_3.jpg
data/Persian/Persian_95.jpg
data/Persian/Persian_138.jpg
data/Persian/Persian_90.jpg
data/Persian/Persian_55.jpg
data/Persian/Persian_115.jpg
data/Persian/Persian_257.jpg
data/Persian/Persian_123.jpg
data/Persian/Persian_258.jpg
data/Persian/Persian_43.jpg
data/Persian/Persian_85.jpg
data/Persian/Persian_186.jpg
data/Persian/Persian_197.jpg
data/Persian/Persian_74.jpg
data/Persian/Persian_72.jpg
data/Persian/Persian_32.jpg
data/Persian/Persian_12.jpg
data/Persian/Persian_61.jpg
data/Persian/Persian_266.jpg
data/Persian/Persian_259.jpg
data/Persian/Persian_140.jpg
data/Persian/Persian_153.jpg
data/Persian/Persian_25.jpg
data/Persian/Persian_53.jpg
data/Persian/Persian_171.jpg
data/Persian/Persian_160.jpg
data/Persian/Persian_256.jpg
data/Persian/Persian_80.jpg
data/Persian/Persian_1.jpg
data/Persian/Persian_83.jpg
data/Persian/Persian_192.jpg
data/P

data/newfoundland/newfoundland_155.jpg
data/newfoundland/newfoundland_30.jpg
data/newfoundland/newfoundland_58.jpg
data/newfoundland/newfoundland_82.jpg
data/newfoundland/newfoundland_111.jpg
data/newfoundland/newfoundland_19.jpg
data/newfoundland/newfoundland_49.jpg
data/newfoundland/newfoundland_87.jpg
data/newfoundland/newfoundland_163.jpg
data/newfoundland/newfoundland_116.jpg
data/newfoundland/newfoundland_166.jpg
data/newfoundland/newfoundland_107.jpg
data/newfoundland/newfoundland_46.jpg
data/newfoundland/newfoundland_69.jpg
data/newfoundland/newfoundland_79.jpg
data/newfoundland/newfoundland_122.jpg
data/newfoundland/newfoundland_72.jpg
data/newfoundland/newfoundland_185.jpg
data/newfoundland/newfoundland_17.jpg
data/newfoundland/newfoundland_18.jpg
data/newfoundland/newfoundland_63.jpg
data/newfoundland/newfoundland_187.jpg
data/newfoundland/newfoundland_132.jpg
data/newfoundland/newfoundland_129.jpg
data/Birman/
data/Birman/Birman_57.jpg
data/Birman/Birman_174.jpg
data/Birman

...

data/american_pit_bull_terrier/american_pit_bull_terrier_181.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_199.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_144.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_126.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_111.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_88.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_169.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_91.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_22.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_124.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_143.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_5.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_166.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_102.jpg
data/american_pit_bull_terrier/american_pit_bull_terrier_125.jpg
data/american_pit_bull_terrier

data/american_bulldog/american_bulldog_69.jpg
data/american_bulldog/american_bulldog_208.jpg
data/american_bulldog/american_bulldog_127.jpg
data/american_bulldog/american_bulldog_198.jpg
data/american_bulldog/american_bulldog_99.jpg
data/american_bulldog/american_bulldog_122.jpg
data/american_bulldog/american_bulldog_56.jpg
data/american_bulldog/american_bulldog_53.jpg
data/american_bulldog/american_bulldog_212.jpg
data/american_bulldog/american_bulldog_101.jpg
data/american_bulldog/american_bulldog_191.jpg
data/american_bulldog/american_bulldog_6.jpg
data/american_bulldog/american_bulldog_75.jpg
data/american_bulldog/american_bulldog_146.jpg
data/american_bulldog/american_bulldog_16.jpg
data/american_bulldog/american_bulldog_167.jpg
data/basset_hound/
data/basset_hound/basset_hound_48.jpg
data/basset_hound/basset_hound_135.jpg
data/basset_hound/basset_hound_99.jpg
data/basset_hound/basset_hound_96.jpg
data/basset_hound/basset_hound_13.jpg
data/basset_hound/basset_hound_134.jpg
data/ba

data/wheaten_terrier/wheaten_terrier_85.jpg
data/wheaten_terrier/wheaten_terrier_119.jpg
data/wheaten_terrier/wheaten_terrier_129.jpg
data/wheaten_terrier/wheaten_terrier_67.jpg
data/wheaten_terrier/wheaten_terrier_110.jpg
data/wheaten_terrier/wheaten_terrier_66.jpg
data/wheaten_terrier/wheaten_terrier_160.jpg
data/wheaten_terrier/wheaten_terrier_43.jpg
data/wheaten_terrier/wheaten_terrier_34.jpg
data/wheaten_terrier/wheaten_terrier_192.jpg
data/wheaten_terrier/wheaten_terrier_26.jpg
data/wheaten_terrier/wheaten_terrier_140.jpg
data/wheaten_terrier/wheaten_terrier_146.jpg
data/wheaten_terrier/wheaten_terrier_117.jpg
data/wheaten_terrier/wheaten_terrier_98.jpg
data/wheaten_terrier/wheaten_terrier_169.jpg
data/wheaten_terrier/wheaten_terrier_185.jpg
data/wheaten_terrier/wheaten_terrier_138.jpg
data/wheaten_terrier/wheaten_terrier_55.jpg
data/wheaten_terrier/wheaten_terrier_1.jpg
data/wheaten_terrier/wheaten_terrier_162.jpg
data/wheaten_terrier/wheaten_terrier_153.jpg
data/wheaten_terrier

data/english_setter/english_setter_179.jpg
data/english_setter/english_setter_98.jpg
data/english_setter/english_setter_21.jpg
data/english_setter/english_setter_39.jpg
data/english_setter/english_setter_146.jpg
data/english_setter/english_setter_119.jpg
data/english_setter/english_setter_32.jpg
data/english_setter/english_setter_64.jpg
data/english_setter/english_setter_135.jpg
data/english_setter/english_setter_145.jpg
data/english_setter/english_setter_181.jpg
data/english_setter/english_setter_105.jpg
data/english_setter/english_setter_51.jpg
data/english_setter/english_setter_161.jpg
data/english_setter/english_setter_148.jpg
data/english_setter/english_setter_130.jpg
data/english_setter/english_setter_100.jpg
data/english_setter/english_setter_157.jpg
data/english_setter/english_setter_17.jpg
data/english_setter/english_setter_47.jpg
data/english_setter/english_setter_103.jpg
data/english_setter/english_setter_79.jpg
data/english_setter/english_setter_66.jpg
data/english_setter/e

data/great_pyrenees/great_pyrenees_39.jpg
data/great_pyrenees/great_pyrenees_93.jpg
data/great_pyrenees/great_pyrenees_198.jpg
data/great_pyrenees/great_pyrenees_163.jpg
data/great_pyrenees/great_pyrenees_48.jpg
data/great_pyrenees/great_pyrenees_120.jpg
data/great_pyrenees/great_pyrenees_20.jpg
data/great_pyrenees/great_pyrenees_11.jpg
data/great_pyrenees/great_pyrenees_22.jpg
data/great_pyrenees/great_pyrenees_57.jpg
data/great_pyrenees/great_pyrenees_138.jpg
data/great_pyrenees/great_pyrenees_168.jpg
data/great_pyrenees/great_pyrenees_133.jpg
data/great_pyrenees/great_pyrenees_175.jpg
data/great_pyrenees/great_pyrenees_164.jpg
data/great_pyrenees/great_pyrenees_169.jpg
data/great_pyrenees/great_pyrenees_24.jpg
data/great_pyrenees/great_pyrenees_144.jpg
data/great_pyrenees/great_pyrenees_25.jpg
data/great_pyrenees/great_pyrenees_139.jpg
data/great_pyrenees/great_pyrenees_176.jpg
data/great_pyrenees/great_pyrenees_104.jpg
data/great_pyrenees/great_pyrenees_174.jpg
data/great_pyrenees/

In [7]:
try:
#     if not os.path.exists('data'):
#         os.makedirs('data')
    if not os.path.exists('dataset'):
        os.makedirs('dataset')
except OSError:
    print("Error to create data directory")

In [8]:
splitfolders.ratio('data', output="dataset", seed=1337, ratio=(.8, .1, .1)) # default values
s3_data_path = 's3://{}/{}'.format(data_bucket, 'oxford_pet_dataset')
!aws s3 cp ./dataset $s3_data_path --recursive

Copying files: 7388 files [00:05, 1256.80 files/s]


upload: dataset/test/Abyssinian/Abyssinian_173.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Abyssinian/Abyssinian_173.jpg
upload: dataset/test/Abyssinian/Abyssinian_190.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Abyssinian/Abyssinian_190.jpg
upload: dataset/test/Abyssinian/Abyssinian_146.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Abyssinian/Abyssinian_146.jpg
upload: dataset/test/Abyssinian/Abyssinian_196.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Abyssinian/Abyssinian_196.jpg
upload: dataset/test/Abyssinian/Abyssinian_21.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Abyssinian/Abyssinian_21.jpg
upload: dataset/test/Abyssinian/Abyssinian_19.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Abyssinian/Abyssinian_19.jpg
upload: dataset/test/Abyssinian/Abyssinian_2.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Abyssinian/Aby

upload: dataset/test/Bombay/Bombay_2.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Bombay/Bombay_2.jpg
upload: dataset/test/Bombay/Bombay_19.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Bombay/Bombay_19.jpg
upload: dataset/test/Bombay/Bombay_15.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Bombay/Bombay_15.jpg
upload: dataset/test/Bombay/Bombay_61.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Bombay/Bombay_61.jpg
upload: dataset/test/Bombay/Bombay_53.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Bombay/Bombay_53.jpg
upload: dataset/test/Bombay/Bombay_206.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Bombay/Bombay_206.jpg
upload: dataset/test/Bombay/Bombay_4.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Bombay/Bombay_4.jpg
upload: dataset/test/Bombay/Bombay_5.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/

upload: dataset/test/Maine_Coon/Maine_Coon_131.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Maine_Coon/Maine_Coon_131.jpg
upload: dataset/test/Maine_Coon/Maine_Coon_145.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Maine_Coon/Maine_Coon_145.jpg
upload: dataset/test/Egyptian_Mau/Egyptian_Mau_59.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Egyptian_Mau/Egyptian_Mau_59.jpg
upload: dataset/test/Egyptian_Mau/Egyptian_Mau_62.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Egyptian_Mau/Egyptian_Mau_62.jpg
upload: dataset/test/Egyptian_Mau/Egyptian_Mau_80.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Egyptian_Mau/Egyptian_Mau_80.jpg
upload: dataset/test/Maine_Coon/Maine_Coon_196.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Maine_Coon/Maine_Coon_196.jpg
upload: dataset/test/Maine_Coon/Maine_Coon_155.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dat

upload: dataset/test/Russian_Blue/Russian_Blue_143.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Russian_Blue/Russian_Blue_143.jpg
upload: dataset/test/Russian_Blue/Russian_Blue_153.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Russian_Blue/Russian_Blue_153.jpg
upload: dataset/test/Russian_Blue/Russian_Blue_179.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Russian_Blue/Russian_Blue_179.jpg
upload: dataset/test/Russian_Blue/Russian_Blue_199.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Russian_Blue/Russian_Blue_199.jpg
upload: dataset/test/Ragdoll/Ragdoll_58.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Ragdoll/Ragdoll_58.jpg
upload: dataset/test/Russian_Blue/Russian_Blue_193.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/test/Russian_Blue/Russian_Blue_193.jpg
upload: dataset/test/Ragdoll/Ragdoll_8.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dat

...

upload: dataset/val/havanese/havanese_191.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/havanese/havanese_191.jpg
upload: dataset/val/havanese/havanese_71.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/havanese/havanese_71.jpg
upload: dataset/val/japanese_chin/japanese_chin_113.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/japanese_chin/japanese_chin_113.jpg
upload: dataset/val/japanese_chin/japanese_chin_126.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/japanese_chin/japanese_chin_126.jpg
upload: dataset/val/japanese_chin/japanese_chin_138.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/japanese_chin/japanese_chin_138.jpg
upload: dataset/val/havanese/havanese_75.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/havanese/havanese_75.jpg
upload: dataset/val/havanese/havanese_9.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/havanese/havanese_9.

upload: dataset/val/miniature_pinscher/miniature_pinscher_113.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/miniature_pinscher/miniature_pinscher_113.jpg
upload: dataset/val/leonberger/leonberger_192.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/leonberger/leonberger_192.jpg
upload: dataset/val/leonberger/leonberger_24.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/leonberger/leonberger_24.jpg
upload: dataset/val/miniature_pinscher/miniature_pinscher_156.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/miniature_pinscher/miniature_pinscher_156.jpg
upload: dataset/val/miniature_pinscher/miniature_pinscher_170.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/miniature_pinscher/miniature_pinscher_170.jpg
upload: dataset/val/leonberger/leonberger_9.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/leonberger/leonberger_9.jpg
upload: dataset/val/miniature_pinscher/miniatu

upload: dataset/val/pomeranian/pomeranian_180.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/pomeranian/pomeranian_180.jpg
upload: dataset/val/pomeranian/pomeranian_35.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/pomeranian/pomeranian_35.jpg
upload: dataset/val/pomeranian/pomeranian_56.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/pomeranian/pomeranian_56.jpg
upload: dataset/val/pomeranian/pomeranian_58.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/pomeranian/pomeranian_58.jpg
upload: dataset/val/pomeranian/pomeranian_18.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/pomeranian/pomeranian_18.jpg
upload: dataset/val/pomeranian/pomeranian_24.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/pomeranian/pomeranian_24.jpg
upload: dataset/val/pomeranian/pomeranian_75.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/pomeranian/pomeranian_75.jpg
uplo

upload: dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_202.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_202.jpg
upload: dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_126.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_126.jpg
upload: dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_160.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_160.jpg
upload: dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_189.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_189.jpg
upload: dataset/val/staffordshire_bull_terrier/staffordshire_bull_terrier_33.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/staffordshire_bull_terr

upload: dataset/val/yorkshire_terrier/yorkshire_terrier_192.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/yorkshire_terrier/yorkshire_terrier_192.jpg
upload: dataset/val/yorkshire_terrier/yorkshire_terrier_191.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/yorkshire_terrier/yorkshire_terrier_191.jpg
upload: dataset/val/yorkshire_terrier/yorkshire_terrier_73.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/yorkshire_terrier/yorkshire_terrier_73.jpg
upload: dataset/val/yorkshire_terrier/yorkshire_terrier_56.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/yorkshire_terrier/yorkshire_terrier_56.jpg
upload: dataset/val/scottish_terrier/scottish_terrier_59.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/scottish_terrier/scottish_terrier_59.jpg
upload: dataset/val/wheaten_terrier/wheaten_terrier_9.jpg to s3://sagemaker-us-east-1-322537213286/oxford_pet_dataset/val/wheaten_terrier/wheaten_terr

## 5. APEX 소개와 Model Source Code 작성

AWS에서 Multigpu distributed training은 기본적으로 Horovod를 사용합니다. 이번에는 Pytorch에서 활용할 수 있는 [APEX](https://github.com/NVIDIA/apex) (A Pytorch EXtension) 패키지를 이용하여 Multigpu distributed training을 수행합니다. APEX 패키지에는 distributed training 기능과 함께 mixed precision training도 할 수 있도록 지원하고 있습니다. 


<p><img src="./imgs/apex.png" width="1100", height="150"></p> 
- mixed precision training (apex.amp) : FP16과 FP32연산을 mixed하여 처리 속도와 정확도를 동시에 잡기 위해 학습을 하는 방법입니다. Tensor Cores에서 FP16를 이용하면 compute 처리량은 8배, 메모리 처리량은 2배 증가하는 반면 메모리 저장은 50% 절감됩니다. (FP : Floating Point, AMP: Automatic Mixed Precision)

- **DataParallel (DP)** : 데이터 샘플의 미니 배치를 여러 개의 더 작은 미니 배치로 나누고 병렬로 작은 미니 배치를 각각 계산하는 방식이며, 단일 host에서 multi-gpu인 경우와 cpu 연산일 경우에 사용합니다. DP의 단점은 GPU가 즐어나면서 communication 비용이 높아지게 되면서 성능저하가 발생하게 되는데 일반적으로 4 gpu 이상일 경우 발생한다고 합니다. 또한, 타 GPU 메모리 대비 0번 GPU 메모리 사용량이 증가하는 현상도 발생합니다.  


- **Distributed Data Parallel (DDP)** : 모듈 수준에서 데이터 병렬 처리를 구현하는 것으로 torch.distributed 패키지의 communication collectives를 사용하여 gradient, parameters, buffers를 동기화합니다. 프로세스 내와 프로세스 간을 사용하는 multi-host의 multi-gpu 와 같은 경우에 사용하게 되는데, 프로세스 내에서는 DDP는 input 모듈을 device_id에 특정한 device로 복제하고, 그에 따라 배치 크기로 input을 분산시키며, outputs는 DataParallel과 유사하게 output_device로 모으게 됩니다.  


- **[Horovod](https://distributed-training-workshop.go-aws.com/)** : MPI 개념을 기반으로 size, rank, local rank, allreduce, allgather, broadcast등을 이용합니다.  


- **[APEX](https://nvidia.github.io/apex)** : apex.parallel.DistributedDataParallel는 모듈 wrapper이며, DDP와 유사합니다. 편리하게 Multi-process를 통한 distributed training을 지원하고, NCCL에 최적화하여 지원합니다.  

#### 1) 학습 스크립트 코딩하기

<p>SageMaker에서 학습하는 것이 아니더라도 실제 모델 아키텍처와 학습을 위한 optimizer와 loss 함수 등을 정의하는 python 파일을 구성하게 됩니다. SageMaker에서 활용하는 python 파일도 동일한 python 파일을 사용하게 됩니다. 연계되는 다른 소스코드 파일이 있는 경우에도 별도 소스코드 수정 없이 학습이 가능하며, SageMaker에서 사용하기 위해서는 기존 python 파일에 SageMaker 학습에 사용할 수 있는 환경변수들만 추가하면 됩니다. 예를 들어, 환경변수 중 <code>SM_MODEL_DIR</code>은 컨테이너 환경에서는 <code>/opt/ml/model</code>를 의미합니다. 다양한 환경변수는 <strong><a href="https://github.com/aws/sagemaker-containers" target="_blank" class ='btn-default'>SageMaker Containers-IMPORTANT ENVIRONMENT VARIABLES</a></strong>를 참고하시기 바랍니다. </p><p>SageMaker 학습이 끝나면 자동은 컨테이너 환경은 삭제가 됩니다. 따라서, 학습이 완료된 모델 산출물과 다양한 output 파일은 S3로 저장해야 합니다. SageMaker는 자동으로 <code>SM_MODEL_DIR</code>에 저장된 최종 모델 파일을 학습이 끝난 다음 model.tar.gz로 압축하여 컨테이너 환경에서 S3의 특정 bucket에 저장하게 됩니다.</p><p> 별도 bucket을 설정하지 않으며, 기본적으로 생성되는 bucket에 저장됩니다. 이외 학습에 이용되는 python source code는 SageMaker 학습이 시작되면서 S3에 저장되며, 별도로 <code>SM_MODEL_DIR</code>에 checkpoint 또는 log 파일을 저장하게 되면 학습이 끝난 이후 자동으로 컨테이너 환경에서 S3로 저장된 파일들이 이동하게 됩니다. 이런 과정을 이해한다면 학습 시 저장되는 다양한 정보들을 저장한 다음 학습이 끝난 후 S3에서 download 받아 활용할 수 있습니다. </p>

<p>아래는 시간 관계 상 미리 작성한 python 학습 스크립트 코드 입니다.</p>

In [9]:
%%writefile ./src_dir/requirements.txt
albumentations
pyarrow
torchnet

Overwriting ./src_dir/requirements.txt


In [75]:
%%writefile ./src_dir/main_trainer.py

import argparse
import json
import logging
import os
import random
import sys
import time
import warnings
import cv2
from typing import Callable, cast

import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torchnet.dataset import SplitDataset


from albumentations import (
    RandomResizedCrop ,HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose, Resize, VerticalFlip, HorizontalFlip, CenterCrop,Normalize
)

import dis_util
import util
        
# print("######### Start Training #########")


logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


class AlbumentationImageDataset(Dataset):
    def __init__(self, image_path, transform, args, check_img=None):
        self.image_path = image_path
        self.transform = transform
        self.args = args
        self.check_img = check_img
        self.image_list =self._loader_file(self.image_path, self.check_img)

    def __len__(self):
        return (len(self.image_list))

    def __getitem__(self, i):
        
        image = cv2.imread(self.image_list[i][0])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Augment an image
        transformed = self.transform(image=image)["image"]
        transformed_image = np.transpose(transformed, (2, 0, 1)).astype(np.float32)
        return torch.tensor(transformed_image, dtype=torch.float), self.image_list[i][1]
    
    def _loader_file(self, image_path, check_img):
        extensions = ('.jpg', '.jpeg', '.png', '.ppm', '.bmp', '.pgm', '.tif', '.tiff', '.webp')
        def is_valid_file(x: str) -> bool:
            return x.lower().endswith(extensions)
        is_valid_file = cast(Callable[[str], bool], is_valid_file)
        
        self.classes = [d.name for d in os.scandir(image_path) if d.is_dir()]
        self.classes.sort()
        self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        
        instances = []
        for target_class in sorted(self.class_to_idx.keys()):
            class_index = self.class_to_idx[target_class]
            target_dir = os.path.join(image_path, target_class)
            if not os.path.isdir(target_dir):
                continue
            for root, _, fnames in sorted(os.walk(target_dir, followlinks=True)):
                for fname in sorted(fnames):
                    path = os.path.join(root, fname)
                    
                    if is_valid_file(path):
                        not_insert = False
                        if check_img:
                            try:
                                image = cv2.imread(path)
                                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                                not_insert = False
                            except:
                                not_insert = True
                                pass
                        if not not_insert:
                            item = path, class_index
                            instances.append(item)
        return instances
    
    
def args_fn():
    parser = argparse.ArgumentParser(description='PyTorch Resnet50 Example')

    # Default Setting
    parser.add_argument('--log-interval', type=int, default=5, metavar='N',
                        help='how many batches to wait before logging training status')
    parser.add_argument('--backend', type=str, default='nccl',
                        help='backend for distributed training (tcp, gloo on cpu and gloo, nccl on gpu)')
    parser.add_argument('--channels-last', type=bool, default=True)
    parser.add_argument('--seed', type=int, default=1, metavar='S',
                        help='random seed (default: 1)')
    parser.add_argument('-p', '--print-freq', default=10, type=int,
                        metavar='N', help='print frequency (default: 10)')

    # Hyperparameter Setting
    parser.add_argument('--model_name', type=str, default='resnet50')
    parser.add_argument('--height', type=int, default=224)
    parser.add_argument('--width', type=int, default=224)
    parser.add_argument('--lr', type=float, default=0.0001)
    parser.add_argument('--num-classes', type=int, default=10)
    parser.add_argument('--num-epochs', type=int, default=3)
    parser.add_argument('--batch-size', type=int, default=64)
    parser.add_argument('--test-batch-size', type=int, default=200, metavar='N',
                        help='input batch size for testing (default: 200)')

    # Setting for Distributed Training
    parser.add_argument('--data_parallel', type=bool, default=False)
    parser.add_argument('--model_parallel', type=bool, default=False)
    parser.add_argument('--apex', type=bool, default=False)
    parser.add_argument('--opt-level', type=str, default='O0')
    parser.add_argument('--keep-batchnorm-fp32', type=str, default=None)
    parser.add_argument('--loss-scale', type=str, default=None)
    parser.add_argument('--sync_bn', action='store_true',
                        help='enabling apex sync BN.')
    parser.add_argument('--prof', default=-1, type=int,
                        help='Only run 10 iterations for profiling.')
    
    # RUBIK Setting for Model Parallel
    parser.add_argument("--num-microbatches", type=int, default=4)
    parser.add_argument("--num-batches", type=int, default=0)
    parser.add_argument("--num-partitions", type=int, default=2)
    parser.add_argument("--horovod", type=int, default=0)
    parser.add_argument("--ddp", type=int, default=0)
    parser.add_argument("--amp", type=int, default=0)
    parser.add_argument("--pipeline", type=str, default="interleaved")
    parser.add_argument("--assert-losses", type=int, default=0)
    parser.add_argument(
        "--partial-checkpoint", type=str, default="", help="The checkpoint path to load"
    )
    parser.add_argument(
        "--full-checkpoint", type=str, default="", help="The checkpoint path to load"
    )
    parser.add_argument(
        "--save-full-model", action="store_true", default=False, help="For Saving the current Model"
    )
    parser.add_argument(
        "--save-partial-model",
        action="store_true",
        default=False,
        help="For Saving the current Model",
    )
    
    

    # SageMaker Container environment
    parser.add_argument('--hosts', type=list,
                        default=json.loads(os.environ['SM_HOSTS']))
    parser.add_argument('--current-host', type=str,
                        default=os.environ['SM_CURRENT_HOST'])
    parser.add_argument('--model-dir', type=str,
                        default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--data-dir', type=str,
                        default=os.environ['SM_CHANNEL_TRAINING'])
    parser.add_argument('--num-gpus', type=int,
                        default=os.environ['SM_NUM_GPUS'])
    parser.add_argument('--output_data_dir', type=str,
                        default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--rank', type=int, default=0)
    
    args = parser.parse_args()    
    return args

def _get_train_data_loader(args, **kwargs):

    transform = Compose([
        RandomResizedCrop(args.height, args.width),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        VerticalFlip(p=0.5),  
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=0.1),
            Blur(blur_limit=3, p=0.1),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomBrightnessContrast(),            
        ], p=0.3),
        HueSaturationValue(p=0.3),
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )
        ], p=1.0)
    
    dataset = AlbumentationImageDataset(
        image_path=os.path.join(args.data_dir, 'train'),
        transform=transform,
        args=args,
        check_img=True
    )
    
    train_sampler = None
    train_dataloader = None
    # If using DDP or Horovod, split the dataset among the model replicas
    if args.model_parallel and args.apex and args.dp_size > 1:
        partitions_dict = {f"{i}": 1 / args.dp_size for i in range(args.dp_size)}
        split_dataset = SplitDataset(dataset, partitions=partitions_dict)
        split_dataset.select(f"{args.dp_rank}")
        
        train_dataloader = data.DataLoader(split_dataset, batch_size=args.batch_size, shuffle=True, **kwargs)
    else:    
        train_sampler = data.distributed.DistributedSampler(
            dataset, num_replicas=int(args.world_size), rank=int(args.rank)) if args.multigpus_distributed else None
        train_dataloader = data.DataLoader(dataset, batch_size=args.batch_size, shuffle=train_sampler is None,
                               sampler=train_sampler, **kwargs)
    return train_dataloader, train_sampler


def _get_test_data_loader(args, **kwargs):
    logger.info("Get test data loader")
    transform = Compose([
        Resize(args.height,args.width),
        Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225],
        )
    ]) 

    dataset = AlbumentationImageDataset(
        image_path=os.path.join(args.data_dir, 'val'),
        transform=transform,
        args=args,
        check_img=True
    )
    return data.DataLoader(dataset, batch_size=args.test_batch_size, shuffle=False)


def train(local_rank, args):
    best_acc1 = -1
    model_history = {}
    model_history = util.init_modelhistory(model_history)
    train_start = time.time()
    print("local_rank : {}".format(local_rank))
    if local_rank is not None:
        args.local_rank = local_rank

    # choose model from pytorch model_zoo
    model = util.torch_model(args.model_name, pretrained=True, num_classes=args.num_classes)  # 1000 resnext101_32x8d
    criterion = nn.CrossEntropyLoss().cuda()

    # distributed_setting
    model, args = dis_util.dist_setting(model, args)
    
    print("******. args.batch_size : {}".format(args.batch_size))

    # CuDNN library will benchmark several algorithms and pick that which it found to be fastest
    cudnn.benchmark = False if args.seed else True

    optimizer = optim.Adam(model.parameters(), lr=args.lr)

    if args.apex:
        model, optimizer, args = dis_util.apex_init(model, optimizer, args)
    elif args.model_parallel:
        model, optimizer, args = dis_util.mp_init(model, optimizer, args)
    elif args.data_parallel:
        model, optimizer, args = dis_util.dp_init(model, optimizer, args)

    train_loader, train_sampler = _get_train_data_loader(args, **args.kwargs)

    logger.info("Processes {}/{} ({:.0f}%) of train data".format(
        len(train_loader.sampler), len(train_loader.dataset),
        100. * len(train_loader.sampler) / len(train_loader.dataset)
    ))


    if args.rank == 0:
        test_loader = _get_test_data_loader(args, **args.kwargs)

        logger.info("Processes {}/{} ({:.0f}%) of test data".format(
            len(test_loader.sampler), len(test_loader.dataset),
            100. * len(test_loader.sampler) / len(test_loader.dataset)
        ))

    for epoch in range(1, args.num_epochs + 1):
        ##
        batch_time = util.AverageMeter('Time', ':6.3f')
        data_time = util.AverageMeter('Data', ':6.3f')
        losses = util.AverageMeter('Loss', ':.4e')
        top1 = util.AverageMeter('Acc@1', ':6.2f')
        top5 = util.AverageMeter('Acc@5', ':6.2f')
        progress = util.ProgressMeter(
            len(train_loader),
            [batch_time, data_time, losses, top1, top5],
            prefix="Epoch: [{}]".format(epoch))

        model.train()
        end = time.time()

        # Set epoch count for DistributedSampler
        if args.multigpus_distributed and not args.model_parallel:
            train_sampler.set_epoch(epoch)

        for batch_idx, (input, target) in enumerate(train_loader):
            #             input = input.cuda()
            #             target = target.cuda()
            input = input.to(args.device)
            target = target.to(args.device)
            batch_idx += 1

#             util.adjust_learning_rate(optimizer, epoch, batch_idx, len(train_loader), args)

            if args.model_parallel:
                output, loss_mb = dis_util.train_step(model, criterion, input, target, args.scaler, args)
                # Rubik: Average the loss across microbatches.
                loss = loss_mb.reduce_mean()
            else:
                output = model(input)
                loss = criterion(output, target)

            # compute gradient and do SGD step
            optimizer.zero_grad()

            if args.apex:
                dis_util.apex_loss(loss, optimizer)
            elif not args.model_parallel:
                loss.backward()
            
            # if args.model_parallel:
            #     if args.amp:
            #         args.scaler.step(optimizer)
            #         args.scaler.update()
            # else:
            
            optimizer.step()

            if args.rank == 0:
#             if args.rank == 0 and batch_idx % args.log_interval == 1:
                # Every print_freq iterations, check the loss, accuracy, and speed.
                # For best performance, it doesn't make sense to print these metrics every
                # iteration, since they incur an allreduce and some host<->device syncs.

                # Measure accuracy
                print("output : {}".format(output.__dict__))
                prec1, prec5 = util.accuracy(output.data, target, topk=(1, 5))

                if args.model_parallel:
                    loss = loss.item()
                else:
                    loss = loss.data
                # Average loss and accuracy across processes for logging
                if args.multigpus_distributed:
                    reduced_loss = dis_util.reduce_tensor(
                        loss, args)
                    prec1 = dis_util.reduce_tensor(prec1, args)
                    prec5 = dis_util.reduce_tensor(prec5, args)
                else:
                    reduced_loss = loss

                # to_python_float incurs a host<->device sync
                losses.update(util.to_python_float(
                    reduced_loss), input.size(0))
                top1.update(util.to_python_float(prec1), input.size(0))
                top5.update(util.to_python_float(prec5), input.size(0))

                # Waiting until finishing operations on GPU (Pytorch default: async)
                torch.cuda.synchronize()
                batch_time.update((time.time() - end)/args.log_interval)
                end = time.time()
                

#                 if args.rank == 0:
                print('Epoch: [{0}][{1}/{2}]  '
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})  '
                      'Speed {3:.3f} ({4:.3f})  '
                      'Loss {loss.val:.10f} ({loss.avg:.4f})  '
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})  '
                      'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                          epoch, batch_idx, len(train_loader),
                          args.world_size*args.batch_size/batch_time.val,
                          args.world_size*args.batch_size/batch_time.avg,
                          batch_time=batch_time,
                          loss=losses, top1=top1, top5=top5))
    
        if args.rank == 0:
            model_history['epoch'].append(epoch)
            model_history['batch_idx'].append(batch_idx)
            model_history['batch_time'].append(batch_time.val)
            model_history['losses'].append(losses.val)
            model_history['top1'].append(top1.val)
            model_history['top5'].append(top5.val)


            acc1 = validate(test_loader, model, criterion,
                            epoch, model_history, args)


            is_best = acc1 > best_acc1
            best_acc1 = max(acc1, best_acc1)

        if not args.multigpus_distributed or (args.multigpus_distributed and args.rank == 0):
            util.save_history(os.path.join(args.output_data_dir,
                                           'model_history.p'), model_history)

            if args.model_parallel:
                dist_util.smp_savemodel(model, optimizer, args)
            else:
                util.save_model({
                    'epoch': epoch + 1,
                    'model_name': args.model_name,
                    'state_dict': model.state_dict(),
                    'best_acc1': best_acc1,
                    'optimizer': optimizer.state_dict(),
                    'class_to_idx': train_loader.dataset.class_to_idx,
                }, is_best, args.model_dir)


def validate(val_loader, model, criterion, epoch, model_history, args):
    batch_time = util.AverageMeter('Time', ':6.3f')
    losses = util.AverageMeter('Loss', ':.4e')
    top1 = util.AverageMeter('Acc@1', ':6.2f')
    top5 = util.AverageMeter('Acc@5', ':6.2f')
    progress = util.ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()
    end = time.time()


    for batch_idx, (input, target) in enumerate((val_loader)):
        #         input = input.cuda()
        #         target = target.cuda()
        input = input.to(args.device)
        target = target.to(args.device)
        batch_idx += 1
        
        # compute output
        with torch.no_grad():
            if args.model_parallel:
                output, loss = dis_util.test_step(
                    model, criterion, input, target)
            else:
                output = model(input)
                loss = criterion(output, target)

        # measure accuracy and record loss
        prec1, prec5 = util.accuracy(output.data, target, topk=(1, 5))

        if args.multigpus_distributed:
            reduced_loss = dis_util.reduce_tensor(loss.data, args)
            prec1 = dis_util.reduce_tensor(prec1, args)
            prec5 = dis_util.reduce_tensor(prec5, args)
        else:
            reduced_loss = loss.data

        losses.update(util.to_python_float(reduced_loss), input.size(0))
        top1.update(util.to_python_float(prec1), input.size(0))
        top5.update(util.to_python_float(prec5), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         print("Validation args.rank : {}".format(args.rank))
        # TODO:  Change timings to mirror train().
        if args.rank == 0:
            print('Test: [{0}/{1}]  '
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})  '
                  'Speed {2:.3f} ({3:.3f})  '
                  'Loss {loss.val:.4f} ({loss.avg:.4f})  '
                  'Prec@1 {top1.val:.3f} ({top1.avg:.3f})  '
                  'Prec@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                      batch_idx, len(val_loader),
                      args.world_size * args.batch_size / batch_time.val,
                      args.world_size * args.batch_size / batch_time.avg,
                      batch_time=batch_time, loss=losses,
                      top1=top1, top5=top5))
            model_history['val_epoch'].append(epoch)
            model_history['val_batch_idx'].append(batch_idx)
            model_history['val_batch_time'].append(batch_time.val)
            model_history['val_losses'].append(losses.val)
            model_history['val_top1'].append(top1.val)
            model_history['val_top5'].append(top5.val)

    print('  Prec@1 {top1.avg:.3f} Prec@5 {top5.avg:.3f}'
          .format(top1=top1, top5=top5))
    model_history['val_avg_epoch'].append(epoch)
    model_history['val_avg_batch_time'].append(batch_time.avg)
    model_history['val_avg_losses'].append(losses.avg)
    model_history['val_avg_top1'].append(top1.avg)
    model_history['val_avg_top5'].append(top5.avg)
    if args.assert_losses:
        dist_util.smp_lossgather(losses.avg, args)
    return top1.avg


def main():
    print("start main function")
    args = args_fn()
    print("args.data_parallel : {} , args.model_parallel : {}, args.apex : {} , args.num_gpus : {}, args.num_classes".format(
        args.data_parallel, args.model_parallel, args.apex, args.num_gpus, args.num_classes))
    
    args.use_cuda = int(args.num_gpus) > 0
    
    args.kwargs = {'num_workers': 8,
                   'pin_memory': True} if args.use_cuda else {}
    args.device = torch.device("cuda" if args.use_cuda else "cpu")
    args = dis_util.dist_init(train, args)
#     train(args)


if __name__ == '__main__':
    main()


Overwriting ./src_dir/main_trainer.py


#### 2) 분산/멀티 gpu에 필요한 code 작성

apex사용과 분산을 위한 init 함수와 분산관련 다양한 함수를 제공합니다. 

In [76]:
%%writefile ./src_dir/dis_util.py

import argparse
import logging
import numpy as np
import os
import random
import sys
import warnings

import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data.distributed
from torch.cuda.amp import autocast

# smdist import package
try:
    # Import smdist PyTorch Modules
    import smdistributed.dataparallel.torch.distributed as dp
    from smdistributed.dataparallel.torch.parallel.distributed import DistributedDataParallel as DDP
    # SMP: Import SMP API
#     import smdistributed.modelparallel.torch as smp 

except ImportError:
    pass
#     raise ImportError("Please install smdist.")


try:
    from apex.parallel import DistributedDataParallel as apexDDP
    import torch.distributed as apex
    from apex.fp16_utils import *
    from apex import amp, optimizers
    from apex.multi_tensor_apply import multi_tensor_applier
except ImportError:
    raise ImportError(
        "Please install apex from https://www.github.com/nvidia/apex to run this example.")


logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


def dist_init(fn, args):
    if args.seed is not None:
        random.seed(args.seed)
        torch.manual_seed(args.seed)
        np.random.seed(args.seed)
        torch.cuda.manual_seed_all(args.seed)
        cudnn.deterministic = True


        if cudnn.deterministic:
            warnings.warn('You have chosen to seed training. '
                          'This will turn on the CUDNN deterministic setting, '
                          'which can slow down your training considerably! '
                          'You may see unexpected behavior when restarting '
                          'from checkpoints.')

    args.is_distributed = len(args.hosts) > 1 and args.backend is not None
    args.is_multigpus = args.num_gpus > 1
    args.multigpus_distributed = (args.is_distributed or args.is_multigpus)

    logger.debug(
        "multigpus_distributed - {}".format(args.multigpus_distributed))
    logger.debug("Number of gpus available - {}".format(args.num_gpus))

#     print("######### Start Training #########")

    if args.multigpus_distributed:
        #         print("channels_last : {}".format(args.channels_last))
        #         if args.channels_last:
        #             args.memory_format = torch.channels_last
        #         else:
        #             args.memory_format = torch.contiguous_format

        if args.apex:
            # Initialize the distributed environment.
            mp.spawn(fn, nprocs=args.num_gpus, args=(args,))
        else:
            if args.data_parallel:
                dp.init_process_group()
            elif args.model_parallel:
                smp.init()

            fn(None, args)
    else:
        fn(0, args)

#     return args


def dist_setting(model, args):
    #     args.data_parallel = False

    print("args.data_parallel : {}".format(args.data_parallel))
    print("args.model_parallel : {}".format(args.model_parallel))
    print("args.apex : {}".format(args.apex))

    args.world_size = 1

    if args.multigpus_distributed:

        args.host_num = args.hosts.index(args.current_host)

        if args.data_parallel:
            args.world_size = dp.get_world_size()
            args.rank = dp.get_rank()   # total rank in all hosts
            args.local_rank = dp.get_local_rank()  # rank per host
        elif args.model_parallel:
            args.world_size = smp.size()
            args.local_rank = smp.local_rank() # rank per host
            args.rank = smp.rank()
            args.dp_size = smp.dp_size()
            args.dp_rank = smp.dp_rank()
            print("smp.rank() : {}, smp.size() : {}, smp.mp_rank() : {}, smp.local_size() : {}, smp.get_mp_group() : {}, smp.get_dp_group() : {}, smp.local_rank() : {}, smp.dp_size() : {}, smp.dp_rank() : {}".format(smp.rank(), smp.size(), smp.mp_rank(), smp.local_size(), smp.get_mp_group(), smp.get_dp_group(), smp.local_rank(), smp.dp_size(), smp.dp_rank()))
        else:
            args.world_size = len(args.hosts) * args.num_gpus
            if args.local_rank is not None:
                args.rank = args.num_gpus * args.host_num + \
                    args.local_rank  # total rank in all hosts

            dist.init_process_group(backend=args.backend,
                                    rank=args.rank, world_size=args.world_size)
            logger.info('Initialized the distributed environment: \'{}\' backend on {} nodes. '.format(
                args.backend, dist.get_world_size()) + 'Current host rank is {}. Number of gpus: {}'.format(
                dist.get_rank(), args.num_gpus))

#     if args.sync_bn:
# #         import apex
#         print("using apex synced BN")
#         model = apex.parallel.convert_syncbn_model(model)
#         # Initialize the distributed environment.
#         os.environ['WORLD_SIZE'] = str(args.world_size)
        print("**** [dist_setting] args.rank : {}".format(args.rank))

        print("args.world_size : {}".format(args.world_size))
        print("Use GPU: {} for training".format(args.local_rank))

        args.lr = args.lr*float(args.world_size)

        args.batch_size //= args.world_size // args.num_gpus
        args.batch_size = max(args.batch_size, 1)

        if args.local_rank is not None:
            torch.cuda.set_device(args.local_rank)

            if not (args.apex or args.data_parallel or args.model_parallel):
                model.cuda(args.local_rank)
                model = torch.nn.parallel.DistributedDataParallel(
                    model, device_ids=[args.rank])
        else:
            if not (args.apex or args.data_parallel or args.model_parallel):
                model.cuda()
                model = torch.nn.parallel.DistributedDataParallel(model)
    elif args.rank is not None:
        torch.cuda.set_device(args.rank)
        if not (args.apex or args.data_parallel or args.model_parallel):
            model = model.cuda(args.rank)
    else:
        if not (args.apex or args.data_parallel or args.model_parallel):
            model = torch.nn.DataParallel(model).cuda()

    return model, args


def apex_init(model, optimizer, args):
    model = model.cuda()
    model, optimizer = amp.initialize(model, optimizer,
                                      opt_level=args.opt_level,
                                      keep_batchnorm_fp32=args.keep_batchnorm_fp32,
                                      loss_scale=args.loss_scale
                                      )
    if args.multigpus_distributed:
        model = apexDDP(model, delay_allreduce=True)
    return model, optimizer, args


def dp_init(model, optimizer, args):
    model = DDP(model.to(args.device), broadcast_buffers=False)
#     model = DDP(model, device_ids=[args.rank], broadcast_buffers=False)
    model.cuda(args.local_rank)
    return model, optimizer, args


def mp_init(model, optimizer, args):
    model = smp.DistributedModel(model)
    args.scaler = smp.amp.GradScaler()
    optimizer = smp.DistributedOptimizer(optimizer)
    if args.partial_checkpoint:
        args.checkpoint = smp.load(args.partial_checkpoint, partial=True)
        model.load_state_dict(args.checkpoint["model_state_dict"])
        optimizer.load_state_dict(args.checkpoint["optimizer_state_dict"])
    elif args.full_checkpoint:
        args.checkpoint = smp.load(args.full_checkpoint, partial=False)
        model.load_state_dict(args.checkpoint["model_state_dict"])
        optimizer.load_state_dict(args.checkpoint["optimizer_state_dict"])

    return model, optimizer, args
def apex_loss(loss, optimizer):
    with amp.scale_loss(loss, optimizer) as scaled_loss:
        scaled_loss.backward()


def reduce_tensor(tensor, args):
    rt = tensor.clone()
#     print("rt : {}".format(rt))
#     dp.all_reduce(rt)
#     print("args.world_size : {}".format(args.world_size))
#     rt /= args.world_size
    return rt


def smp_lossgather(loss, args):
    if args.use_horovod or args.use_ddp:
        # Rubik: If using data parallelism, gather all losses across different model
        # replicas and check if losses match.

        losses = smp.allgather(loss, smp.DP_GROUP)
        for l in losses:
            assert math.isclose(l, losses[0])

        assert loss < 0.14
    else:
        assert loss < 0.08


def smp_savemodel(model, optimizer, args):
    smp.barrier()
    if args.dp_rank == 0:
        model_dict = model.local_state_dict()
        opt_dict = optimizer.local_state_dict()
        smp.save(
            {"model_state_dict": model_dict, "optimizer_state_dict": opt_dict},
            f"/opt/ml/local_checkpoints/model_checkpoint.pt",
            partial=True,
        )
    smp.barrier()
    
    if args.save_full_model:
        if args.dp_rank == 0:
            model_dict = model.state_dict()
            opt_dict = optimizer.state_dict()
            smp.save(
                {"model_state_dict": model_dict, "optimizer_state_dict": opt_dict},
                "./pt_mnist_checkpoint.pt",
                partial=False,
            )
    # Waiting the save checkpoint to be finished before run another allgather_object
    smp.barrier()

    return model, optimizer, args

# Rubik: Define smp.step. Return any tensors needed outside.
# @smp.step
def train_step(model, criterion, input, target, scaler, args):
    with autocast(1 > 0):
        output = model(input)

    loss = criterion(output, target)

    # scaled_loss = scaler.scale(loss) if args.amp else loss
    model.backward(loss)
    return output, loss


# Rubik: Define smp.step for evaluation.
# @smp.step
def test_step(model, criterion, input, target):
    output = model(input)
    # sum up batch loss
    loss = criterion(output, target, reduction="sum").item()
    return output, loss


Overwriting ./src_dir/dis_util.py


#### 3) 모델 학습에 공통적으로 사용할 수 있는 util 코드 작성

In [77]:
%%writefile ./src_dir/util.py

import codecs
import json
import logging
import os
import shutil
import sys
import time


import torch
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data.distributed
from torchvision import models

from collections import OrderedDict

# import sagemaker_containers

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler(sys.stdout))


def torch_model(model_name, pretrained=True, num_classes=0):
    model_names = sorted(name for name in models.__dict__
                         if name.islower() and not name.startswith("__")
                         and callable(models.__dict__[name]))

    if(model_name == "inception_v3"):
        raise RuntimeError(
            "Currently, inception_v3 is not supported by this example.")

    # create model
    if pretrained:
        print("=> using pre-trained model '{}'".format(model_name))
        model = models.__dict__[model_name](pretrained=True)
    else:
        print("=> creating model '{}'".format(model_name))
        model = models.__dict__[model_name]()

    if num_classes > 0:
        n_inputs = model.fc.in_features

        # add more layers as required
        classifier = nn.Sequential(OrderedDict([
            ('fc', nn.Linear(n_inputs, num_classes))
        ]))

        model.fc = classifier

    return model


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


def save_model(state, is_best, model_dir):
    logger.info("Saving the model.")
    filename = os.path.join(model_dir, 'checkpoint.pth')
    # recommended way from http://pytorch.org/docs/master/notes/serialization.html
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, os.path.join(model_dir, 'model_best.pth'))


class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def adjust_learning_rate(optimizer, epoch, step, len_epoch, args):
    """LR schedule that should yield 76% converged accuracy with batch size 256"""
    factor = epoch // 30

    if epoch >= 80:
        factor = factor + 1

    lr = args.lr*(0.1**factor)

    """Warmup"""
    if epoch < 5:
        lr = lr*float(1 + step + epoch*len_epoch)/(5.*len_epoch)

    if args.rank == 0:
        print("epoch = {}, step = {}, lr = {}".format(epoch, step, lr))

    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def save_history(path, history):

    history_for_json = {}
    # transform float values that aren't json-serializable
    for key in history.keys():
        history_for_json[key] = list(map(float, history[key]))

    with codecs.open(path, 'w', encoding='utf-8') as f:
        json.dump(history_for_json, f, separators=(
            ',', ':'), sort_keys=True, indent=4)


def to_python_float(t):
    if hasattr(t, 'item'):
        return t.item()
    else:
        return t[0]


def init_modelhistory(model_history):
    model_history['epoch'] = []
    model_history['batch_idx'] = []
    model_history['batch_time'] = []
    model_history['losses'] = []
    model_history['top1'] = []
    model_history['top5'] = []
    model_history['val_epoch'] = []
    model_history['val_batch_idx'] = []
    model_history['val_batch_time'] = []
    model_history['val_losses'] = []
    model_history['val_top1'] = []
    model_history['val_top5'] = []
    model_history['val_avg_epoch'] = []
    model_history['val_avg_batch_time'] = []
    model_history['val_avg_losses'] = []
    model_history['val_avg_top1'] = []
    model_history['val_avg_top5'] = []
    return model_history

Overwriting ./src_dir/util.py


## 6.Hyperparameter 셋팅

In [85]:
hyperparameters = {
        'model_name' : 'resnext101_32x8d',
        'num-classes' : 37,
        'height' : 128,
        'width' : 128,
        'num-epochs': 10,
        'batch-size' : 128, # 128*4,
        'lr': 0.00001,
#         'model_parallel' : True,
        'data_parallel' : True,
#         'apex' : True
    }

In [86]:
train_instance_type = 'ml.p3.16xlarge'
image_uri = None
distribution = None
train_job_name = 'single'

if hyperparameters.get('data_parallel'):
    train_job_name = 'dp-dist'
#     image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu18.04"
    distribution = {"smdistributed": {
                        "dataparallel": {
                                "enabled": True
                        }
                   }
                 }
elif hyperparameters.get('model_parallel'):
    train_job_name = 'mp-dist'
#     image_uri = "763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:1.6.0-gpu-py36-cu110-ubuntu18.04"

    mpioptions = "-verbose -x orte_base_help_aggregate=0 "
    mpioptions += "--mca btl_vader_single_copy_mechanism none "
    distribution={
      "smdistributed": {
          "modelparallel": {
              "enabled":True,
              "parameters": {
                  "microbatches": 4,
                  "placement_strategy": "spread",
                  "pipeline": "interleaved",
                  "optimize": "speed",
                  "partitions": 2,
                  "ddp": True,
              }
          }
      },
      "mpi": {
            "enabled": True,
            "processes_per_host": 2, # Pick your processes_per_host
            "custom_mpi_options": mpioptions 
      },
    }

elif hyperparameters.get('apex'):
    train_job_name = 'apex'
#     mpioptions = "-verbose -x orte_base_help_aggregate=0 "
#     mpioptions += "--mca btl_vader_single_copy_mechanism none "
#     distribution={
#       "mpi": {
#             "enabled": True,
#             "processes_per_host": 8, # Pick your processes_per_host
# #             "custom_mpi_options": mpioptions 
#       },
#     }
    
else:
    train_instance_type = 'ml.p3.2xlarge'

train_instance_count = 1
print("train_job_name : {} \ntrain_instance_type : {} \nimage_uri : {} \ndistribution : {}".format(train_job_name, train_instance_type, image_uri, distribution))    

train_job_name : dp-dist 
train_instance_type : ml.p3.16xlarge 
image_uri : None 
distribution : {'smdistributed': {'dataparallel': {'enabled': True}}}


In [87]:
from sagemaker.local import LocalSession

if train_instance_type == 'local':
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    s3_data_path='file:///home/ec2-user/SageMaker/Sagemaker-HOL/3.Distributed_training_and_EIA_Deploy_for_Pytorch/dataset'
else:
    sagemaker_session = sagemaker.Session(sagemaker_client=sm)
    s3_data_path = 's3://{}/{}'.format(data_bucket, 'oxford_pet_dataset')

## 7. `Pytorch` estimator를 이용한 training job 생성하기


<p><strong><code>sagemaker.pytorch.PyTorch</code></strong> estimator는 처음 실행하는 스크립트 위치와 다양한 연계 코드들이 위치한 디렉토리 정보를 찾아서 스크립트를 S3에 upload하고 SageMaker의 training job을 수행하게 됩니다. training job은 학습을 수행한 단위입니다. 학습을 1번 돌리면 training job이 1개 생성됩니다. 몇 가지 중요 파라미터를 아래와 같이 설명드립니다. </p>

- **entry_point** : 학습을 처음 실행하는 Python 소스 파일의 절대 또는 상대 경로이며, source_dir이 지정된 경우 entry_point는 source_dir 내 파일이 됩니다.
- **source_dir** : 학습에 연계되는 다양한 소스코드 파일이 들어 있는 디렉토리 위치이며, 절대, 상대 경로 또는 S3 URI가 모두 가능하며,source_dir이 S3 URI 인 경우 tar.gz 파일이 됩니다.
- **role** : Amazon SageMaker가 사용자를 대신해 작업(예: S3 버킷에서 모델 결과물이라고 하는 훈련 결과 읽기 및 Amazon S3에 훈련 결과 쓰기)을 수행하는 AWS Identity and Access Management(IAM) 역할입니다.
- **train_instance_count** : 학습을 수행하는 instance 개수를 정의할 수 있습니다.
- **train_instance_type** : 학습을 수행하는 instance 타입을 정의할 수 있습니다.
- **train_volume_size** : 학습 인스턴스에 연결할 Amazon Elastic Block Store(Amazon EBS) 스토리지 볼륨의 크기(GB)입니다. File 모드를 사용할 경우 이 값이 훈련 데이터를 충분히 저장할 수 있는 크기여야 합니다(File 모드가 기본값)
- **train_use_spot_instances** : 학습에서 SageMaker Managed Spot 인스턴스를 사용할지 여부를 지정합니다. 활성화되면 train_max_wait도 설정해야 합니다.
- **train_max_run** : 최대 학습 시간을 설정할 수 있으며, 이 시간이 지나면 Amazon SageMaker는 현재 상태에 관계없이 작업을 종료합니다. (기본값 : 24 * 60 * 60)
- **train_max_wait** : SageMaker Managed Spot 인스턴스를 기다리는 초 단위의 시간을 의미하는 것으로, 이 시간이 지나면 Amazon SageMaker는 스팟 인스턴스가 사용 가능해지기를 기다리는 것을 중지하며 결과는 fail이 됩니다.
- **framework_version** : 학습에 사용될 특정 Pytorch 버전을 정의할 수 있습니다.
- **py_version** : 컨테이너 환경이 python3일 경우 py3, python2일 경우 py2로 설정하면 됩니다. python2는 지원이 중단되었지만 기존 python2로 구성된 파일들을 지원하기 위해 현재 계속 사용할 수 있습니다. 없을 경우에는 기본적으로 py3 입니다.
- **hyperparameters** : 학습에 사용할 하이퍼 파라미터를 정의할 수 있으며, 정의된 하이퍼 파라미터 값들은 모두 학습 컨테이너로 전송이 됩니다.

<p> 추가적으로 분산/ 멀티 GPU 학습도 가능합니다. SageMaker는 <strong><a href="https://github.com/horovod/horovod" target="_blank" class ='btn-default'>Horovod</a></strong>에 최적화된 환경을 제공하고 있으며, Pytorch의 경우 1.5.0부터 기본 docker에서 apex를 지원합니다.</p>


In [88]:
%%time

# all input configurations, parameters, and metrics specified in estimator 
# definition are automatically tracked
estimator = PyTorch(
    entry_point='./main_trainer.py',
    source_dir='./src_dir',
    role=role,
    sagemaker_session=sagemaker_session,
    framework_version='1.6.0',
    py_version='py3',
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    volume_size=400,
    hyperparameters=hyperparameters,
    distribution=distribution,
#     train_use_spot_instances=True,  # spot instance 활용
#     train_max_run=12*60*60,
#     train_max_wait=12*60*60
)

CPU times: user 95 µs, sys: 22 µs, total: 117 µs
Wall time: 121 µs


## 8. Fit 함수로 학습 시작하기 

<p>학습을 시작하는 것은 <strong><code>estimator.fit (training_data_uri)</code></strong>이 호출되는 경우입니다. 여기에서 실제 데이터가 있는 S3의 위치가 입력으로 사용됩니다. <code>fit</code>에서는 <code>training</code>라는 기본 채널을 생성하며, 이 위치의 데이터는 S3에서 실제 컨테이너 환경에서는 <code>SM_CHANNEL_TRAINING</code> 위치로 복사되어 학습에 활용이 가능합니다. <code>fit</code>은 몇 가지 다른 유형의 입력도 허용하는데 자세한 내용은 <strong><a href="https://sagemaker.readthedocs.io/en/stable/estimators.html#sagemaker.estimator.EstimatorBase.fit" target="_blank" class ='btn-default'>API 문서</a></strong>를 참고하실 수 있습니다.</p>
<p> 학습이 시작되면 Tensorflow 컨테이너에서는 <code>image_classifier.py</code>를 실행되며, <code>Estimator</code>에서 <code>hyperparameters</code> 와 <code>model_dir</code>을 스크립트의 파라미터로 전달합니다. <code>model_dir</code>을 별도로 전달하지 않으며, 기본값은<strong>s3://[DEFAULT_BUCKET]/[TRAINING_JOB_NAME] </strong>이 되며 실제 스크립트 실행은 다음과 같습니다. </p>
    
```bash
python image_classifier.py --model_dir s3://[DEFAULT_BUCKET]/[TRAINING_JOB_NAME]
```
<p>학습이 완료되면 training job은 Tensorflow serving을 위해 saved model을 S3에 upload합니다.</p>
<p><code>fit</code>에서 <strong>wait=True</strong>로 설정할 경우 <strong>Synchronous</strong> 방식으로 동직하게 되며, <strong>wait=False</strong>일 경우 <strong>Aynchronous</strong> 방식으로 동작되어 여러 개의 Training job을 동시에 실행할 수 있습니다. </p>

In [89]:
training_job_name = "{}-dis-training-job-{}".format(train_job_name, int(time.time()))


# Now associate the estimator with the Experiment and Trial
estimator.fit(
    inputs={'training': s3_data_path}, 
    job_name=training_job_name,
    logs='All',
    wait=False,
)

In [90]:
job_name=estimator.latest_training_job.name

<p><strong>Aynchronous</strong>로 진행된 Training job은 아래와 같은 방법으로 진행상황을 실시간으로 확인할 수 있습니다.</p>

In [ ]:
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

2020-12-10 14:52:30 Starting - Starting the training job...
2020-12-10 14:52:55 Starting - Launching requested ML instancesProfilerReport-1607611950: InProgress
....

<p>학습이 모두 완료된 다음에 S3에서 모델 산출물을 SageMaker Notebook 환경으로 내려받습니다.</p>

In [ ]:
artifacts_dir = estimator.model_data.replace('model.tar.gz', '')
print(artifacts_dir)
!aws s3 ls --human-readable {artifacts_dir}

In [ ]:
model_dir = './model'
output_dir = './output'

In [ ]:
!rm -rf $model_dir
!rm -rf $output_dir

In [ ]:
import json , os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
!aws s3 cp {artifacts_dir}model.tar.gz {model_dir}/model.tar.gz
!tar -xzf {model_dir}/model.tar.gz -C {model_dir}
!aws s3 cp {artifacts_dir}output.tar.gz {output_dir}/output.tar.gz
!tar -xzf {output_dir}/output.tar.gz -C {output_dir}

## 9. 결과 확인

In [ ]:
import json , os

with open(os.path.join(output_dir, 'model_history.p'), "r") as f:
    model_history = json.load(f)

In [ ]:
import matplotlib.pyplot as plt

def plot_training_curves(history): 
    
    fig, axes = plt.subplots(1, 4, figsize=(18, 4), sharex=True)
    
    ax = axes[0]
    ax.plot(history['epoch'], history['losses'], label='train')
    ax.plot(history['val_avg_epoch'], history['val_avg_losses'], label='validation')
    ax.set(
        title='model loss',
        ylabel='loss',
        xlabel='epoch')
    ax.legend()
    
    ax = axes[1]
    ax.plot(history['epoch'], history['batch_time'], label='train')
    ax.plot(history['val_avg_epoch'], history['val_avg_batch_time'], label='validation')
    ax.set(
        title='model batch_time',
        ylabel='batch_time',
        xlabel='epoch')
    ax.legend()
    
    
    ax = axes[2]
    ax.plot(history['epoch'], history['top1'], label='train')
    ax.plot(history['val_avg_epoch'], history['val_avg_top1'], label='validation')
    ax.set(
        title='top1 accuracy',
        ylabel='accuracy',
        xlabel='epoch')
    ax.legend()
    
    ax = axes[3]
    ax.plot(history['epoch'], history['top5'], label='train')
    ax.plot(history['val_avg_epoch'], history['val_avg_top5'], label='validation')
    ax.set(
        title='top5 accuracy',
        ylabel='accuracy',
        xlabel='epoch')
    ax.legend()
    fig.tight_layout()
    
plot_training_curves(model_history)

In [ ]:
%store hyperparameters model_dir output_dir artifacts_dir

<p></p>
<p>Amazon SageMaker에서 모든 학습을 완료하였습니다. </p>